In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.helpers import launch_env, wrap_env, view_results_ipython, change_exercise, seedall, force_done, evaluate_policy
from utils.helpers import SteeringToWheelVelWrapper, ResizeWrapper, ImgWrapper

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

INFO:aido-protocols:aido-protocols 5.0.5
21:08:52|zn|__init__.py:6|<module>(): zn 2.0.3
21:08:53|zj|__init__.py:5|<module>(): zj 2.0.4
21:08:53|gym-duckietown|__init__.py:10|<module>(): gym-duckietown 5.0.3

21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop_cw-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-straight_road-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_obstacles-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-zigzag_dists-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-udem1-v0
21:08:53|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop-v0

# Reinforcement Learning Basics

Reinforcement Learning, as we saw in lecture, is the idea of learning a _policy_ in order to maximize future (potentially discounted) rewards. Our policy, similar to the imitation learning network, maps raw image observations to wheel velocities, and at every timestep, receives a _reward_ from the environment. 

Rewards can be sparse (`1` if goal or task is completed, `0` otherwise) or dense; in general, dense rewards make it easier to learn policies, but as we'll see later in this exercise, defining the correct dense reward is an engineering challenge on its own.

Today's reinforcement learning algorithms are often a mix between _value-based_ and _policy-gradient_ algorithms, instances of what is called an _actor-critic_ formulation. Actor-critic methods have had a lot of research done on them in recent years (especially within in the deep reinforcement learning era), and later in this exercise, we shall also rediscover the formulation's original problems and different methods currently used to stabilize learning.

We begin by defining two networks, an `Actor` and `Critic`; in this exercise, we'll be using a deep RL algorithm titled _Deep Deterministic Policy Gradients_. 

## Reward Engineering

In this part of the exercise, we will experiment with the reward formulation. Given the same model, we'll see how the effect of various reward functions changes the final policy trained. 

In the section below, we'll take a look at the reward function implemented in `gym-duckietown` with a slightly modified training loop. Traditionally, we `reset()` the environment to start an episode, and then `step()` the environment forward for a set amount of time, executing a new action. If this sounds a bit odd, especially for roboticists, you're right - in real robotics, most code runs asynchronously. As a result, although `gym-duckietown` runs locally by stopping the environment, the `AIDO` submissions will run asynchronously, executing the same action until a new one is received.

In [2]:
def updated_reward(env, c1=1.0, c2=-10, c3=40):
    # Compute the collision avoidance penalty
    pos, angle, speed = env.cur_pos, env.cur_angle, env.speed
    col_penalty = env.proximity_penalty2(pos, angle)
    
    # Get the position relative to the right lane tangent
    try:
        lp = env.get_lane_pos2(pos, angle)
    except NotInLane:
        reward = c3 * col_penalty
    else:
        # Compute the reward
        reward = (
                c1 * speed * lp.dot_dir +
                c2 * np.abs(lp.dist) +
                c3 * col_penalty
        )
    return reward

In [3]:
nepisodes = 3

In [4]:
local_env = launch_env()
local_env = wrap_env(local_env)
local_env = ResizeWrapper(local_env)
local_env = ImgWrapper(local_env)

# actor = Actor(action_dim=2, max_action=1.0)

for _ in range(nepisodes):
    done = False
    obs = local_env.reset()
    obs = torch.from_numpy(obs)
    
    c1 = 16
    c2 = -10
    c3 = 40
    
    while not done:
        action = np.random.random(2)
#         action = actor(obs)        
        obs, r, done, info = local_env.step(action)
#         obs = torch.from_numpy(obs)

        new_r = updated_reward(local_env, c1, c2, c3)
        print(r, new_r)
 

21:08:55|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
21:08:55|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
21:08:55|gym-duckietown|simulator.py:550|_load_map(): loading map file "/duckietown/simulation/gym_duckietown/maps/loop_empty.yaml"
21:08:55|gym-duckietown|objmesh.py:50|__init__(): loading mesh "duckiebot.obj"
21:08:55|gym-duckietown|objmesh.py:238|_load_mtl(): loading materials from "/duckietown/simulation/gym_duckietown/meshes/duckiebot.mtl"
21:08:55|gym-duckietown|objmesh.py:50|__init__(): loading mesh "duckie.obj"
21:08:55|gym-duckietown|graphics.py:60|load_texture(): loading texture "duckie.png"
21:08:56|gym-duckietown|objmesh.py:50|__init__(): loading mesh "cone.obj"
21:08:56|gym-duckietown|graphics.py:60|load_texture(): loading texture "cone.png"
21:08:56|gym-duckietown|graphics.py:60|load_texture(): loading texture "floor_1.png"
21:08:56|gym-duckietown|graphics

21:08:56|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
21:08:56|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
21:08:56|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.59826268 0.         2.08721855]
21:08:56|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.5894849  0.         2.18432262]
21:08:56|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.60704045 0.         1.99011448]
21:08:56|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.48173779 0.         2.07668522]
21:08:56|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.48172154 0.         1.84866924] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f5728a42278>, 'color': array([1, 1, 1])}
21:08:56|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

21:08:56|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [4.02468139 0.         1.75868687]
21:08:56|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.49620504 0.         2.20742184] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f5728a42278>, 'color': array([1, 1, 1])}
21:08:56|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
21:08:56|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
21:08:56|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.57500004 0.         2.14999713]
21:08:56|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.49620504 0.         2.20742184]
21:08:56|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.65379505 0.         2.09257242]
21:08:56|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.50609039 0.         2.05

21:09:00|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
21:09:00|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
21:09:00|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [2.33703727 0.         2.57700384]
21:09:00|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [2.24568806 0.         2.542922  ]
21:09:00|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [2.42838647 0.         2.61108568]
21:09:00|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [2.37793548 0.         2.46738479]
21:09:00|gym-duckietown|simulator.py:1107|_drivable_pos(): [2.28873639 0.         2.68406941] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f5728a42278>, 'color': array([1, 1, 1])}
21:09:00|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.10994753896554466 -1.3098857241571675
-0.10968448715635493 -1.3096794463677122
-0.10951339384152226 -1.309471538968344
-0.10925101288990735 -1.3090443315818474
-0.10840354841804478 -1.3082176817999824


...

0.7759224676685657 -0.40997333561276106
0.7380595173453964 -0.43127978005227197
0.6877763255040639 -0.46036176485426084
0.6275524995498525 -0.4959154790142229
0.5498303670525224 -0.54111656875872
0.4670964858804718 -0.5910139115494022
0.37144083815558726 -0.648545700542224
0.2788010609378865 -0.7075767035833089
0.19352171494565562 -0.7657645821189218
0.09847629984643924 -0.8253545579705344
0.007133599190610407 -0.8837848012775114
-0.09838595369219372 -0.9539802307280209
-0.22282034196629652 -1.0352936915017528
-0.31682116969947616 -1.113770123119541
-0.4601725967807134 -1.2013582883310887
-0.5687288832716886 -1.28739416956885
-0.6881204099022901 -1.382879176205683


21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.595193171900072, 0, 2.6643545670983197] angle -1.9298338259152519
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5909440009632916, 0, 2.675813844521206] angle -1.921936508772683
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5869227181463175, 0, 2.6871579647444612] angle -1.9009797777884352
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5832472360587273, 0, 2.698034661040131] angle -1.8923633586628599
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.579704709406122, 0, 2.708991966730643] angle -1.874621286124725
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5764038008694135, 0, 2.7200439272414987] angle -1.8474469353014031
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5733093828994877, 0, 2.731421458102658] angle -1.8252514585062445
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5706243907135855, 0

-0.8136481488853123 -1.477570001741049
-0.9309582659032917 -1.569351171844413
-1.0600008124258997 -1.673778986015617
-1.2030033291886704 -1.7784743154341065
-1.3299682109080566 -1.87992837800459
-1.4629836111739327 -1.9830253224315164
-1.6084308542048522 -2.0879610856244173
-1.7527976162738743 -2.196877432127967
-1.8991271162275558 -2.3021802739887662
-2.0543277963057562 -2.41736284669075
-2.215529064775749 -2.538848437565707
-2.3862069469699647 -2.6602609085481506
-2.554779497918785 -2.7825557814393505


21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5605643415342882, 0, 2.8135580810558514] angle -1.6225481191460653
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5600148720217946, 0, 2.8259857730252858] angle -1.607413621099599
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.5596138569404454, 0, 2.8374804193613334] angle -1.6039249811776743
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.559251939957646, 0, 2.8491900224253834] angle -1.5994634142565025
21:09:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.558984445254509, 0, 2.859051709066204] angle -1.5963652212890949
21:09:05|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.55729708 0.         2.92503014] corresponds to tile at (6, 5) which is not drivable: {'coords': (6, 5), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f5728a42278>, 'color': array([1, 1, 1])}
21:09:05|gym-duckieto

-2.696806788444103 -2.8873142772250477
-2.8407252372837357 -3.0010660138273346
-2.9875457379852657 -3.1244729167873118
-3.1059934798215845 -3.2387608088267306
-3.2277771252077176 -3.3552221397110578
-1000 -3.453372796000826


In [5]:
view_results_ipython(local_env)

**Question 0: After understanding the above computed reward, experiment with the constants for each component. What type of behavior does the above reward function penalize? Is this good or bad in context of autonomous driving? Name some other issues that can arise with single-objective optimization. In addition, give three sets of constants and explain qualitatively what types of behavior each penalizes or rewards (note, you may want to use a different action policy than random)**. Place the answers to the above in `reinforcement-learning-answers.txt`




# The Reinforcement Learning Learning Code

Below we'll see a relatively naive implementation of the actor-critic training loop, which proceeds as follows: the critic is tasked with a supervised learning problem of fitting rewards acquired by the agent. Then, the policy, using policy gradients, maximizes the return according to the critic's estimate, rather than using Monte-Carlo updates.

Below, we see an implementation of `DDPGAgent`, a class which handles the networks and training loop. 

In [6]:
!pip install ipywidgets --quiet
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Traceback (most recent call last):
  File "/usr/local/bin/jupyter", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/command.py", line 134, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-labextension` not found.


In [7]:
class Actor(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Actor, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.1)

        self.lin1 = nn.Linear(flat_size, 100)
        self.lin2 = nn.Linear(100, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = self.bn1(self.relu(self.conv1(x)))
        x = self.bn2(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = self.relu(self.lin1(x))

        x = self.lin2(x)
        x = self.max_action * self.tanh(x)
        
        return x
    
class Critic(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Critic, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.1)

        self.lin1 = nn.Linear(flat_size + action_dim, 100)
        self.lin2 = nn.Linear(100, action_dim)

        self.max_action = max_action

    def forward(self, obs, action):
        x = self.bn1(self.relu(self.conv1(obs)))
        x = self.bn2(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = torch.cat([x, action], 1)
        x = self.relu(self.lin1(x))

        x = self.lin2(x)
        x = self.max_action * self.tanh(x)
        
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class DDPGAgent(object):
    def __init__(self, state_dim, action_dim, max_action=1.0):
        super(DDPGAgent, self).__init__()
        self.flat = False

        self.actor = Actor(action_dim, max_action).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=5e-2)
        
        self.critic = Critic(action_dim, max_action).to(device) # CriticCNN -> Critic
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=5e-2)

    def predict(self, state):
        assert state.shape[0] == 3
        state = torch.FloatTensor(np.expand_dims(state, axis=0)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer, iterations, batch_size=64, discount=0.99):
        for it in range(iterations):
            # Sample replay buffer
            sample = replay_buffer.sample(batch_size, flat=self.flat)
            state = torch.FloatTensor(sample["state"]).to(device)
            action = torch.FloatTensor(sample["action"]).to(device)
            next_state = torch.FloatTensor(sample["next_state"]).to(device)
            # NOTE: why 1 - done?
            done = torch.FloatTensor(1 - sample["done"]).to(device) 
            reward = torch.FloatTensor(sample["reward"]).to(device)
            
            # Compute the target Q value
            target_Q = self.critic(next_state, self.actor(next_state))
            
            # TODO: - no detach is a subtle, but important bug!
            target_Q = reward + (done * discount * target_Q)

            # Get current Q estimate
            current_Q = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q, target_Q)

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Compute actor loss
            actor_loss = -self.critic(state, self.actor(state)).mean()
            
            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
            
            print(f"Iteration {i}, critic_loss: {critic_loss}, actor_loss: {actor_loss}")
            
    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '{}/{}_actor.pth'.format(directory, filename))
        torch.save(self.critic.state_dict(), '{}/{}_critic.pth'.format(directory, filename))

    def load(self, filename, directory):
        self.actor.load_state_dict(torch.load('{}/{}_actor.pth'.format(directory, filename), map_location=device))
        self.critic.load_state_dict(torch.load('{}/{}_critic.pth'.format(directory, filename), map_location=device))


You'll notice that the training loop needs a `replay_buffer` object. In value-based and actor-critic methods in deep reinforcement learning, the use of a replay buffer is crucial. In the following sections, you'll explore why this is the case, and some other stabilization techniques that are needed in order to get the above code to work. Below, you can find an implementation of the replay buffer, as well the training loop that we use to train DDPG.

In [9]:
from dataclasses import dataclass
import typing
from typing import *

class ReplayBufferEntry(NamedTuple):
    state: np.ndarray
    next_state: np.ndarray
    action: np.ndarray
    reward: float
    done: float
    

# Simple replay buffer
class ReplayBuffer(object):
    def __init__(self, max_size=1e6):
        self.storage = []
        self.max_size = max_size

    # Expects tuples of (state, next_state, action, reward, done)
    def add(self, state, next_state, action, reward, done):
        if len(self.storage) < self.max_size:
            self.storage.append(ReplayBufferEntry(state, next_state, action, reward, done))
        else:
            # Remove random element in the memory beforea adding a new one
            self.storage.pop(random.randrange(len(self.storage)))
            self.storage.append(ReplayBufferEntry(state, next_state, action, reward, done))


    def sample(self, batch_size=100, flat=True):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, next_states, actions, rewards, dones = [], [], [], [], []

        for i in ind:
            state, next_state, action, reward, done = self.storage[i]

            if flat:
                states.append(np.array(state, copy=False).flatten())
                next_states.append(np.array(next_state, copy=False).flatten())
            else:
                states.append(np.array(state, copy=False))
                next_states.append(np.array(next_state, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(np.array(reward, copy=False))
            dones.append(np.array(done, copy=False))

        # state_sample, action_sample, next_state_sample, reward_sample, done_sample
        return {
            "state": np.stack(states),
            "next_state": np.stack(next_states),
            "action": np.stack(actions),
            "reward": np.stack(rewards).reshape(-1,1),
            "done": np.stack(dones).reshape(-1,1)
        }

In [12]:
seed_ = 123
max_timesteps = 500 
batch_size = 64
discount = 0.99
eval_freq = 5e3
file_name = 'dt-class-rl'
start_timesteps = 10e3
start_timesteps = 100
expl_noise = 0.1

env_timesteps = 300
save_models = True

In [14]:
import os
import logging
import contextlib
logger = logging.getLogger("gym-duckietown")
logging.basicConfig(level="ERROR")


@contextlib.contextmanager
def temporarily_disable_logging(logger):
    start_value = logger.disabled
    try:
        logger.disabled = True
        yield
    except UserWarning:
        pass
    finally:
        logger.disabled = start_value

with temporarily_disable_logging(logger):
    local_env = launch_env()
    # local_env = wrap_env(local_env)
    local_env = ResizeWrapper(local_env)
    local_env = ImgWrapper(local_env)

    if not os.path.exists("./pytorch_models"):
        os.makedirs("./pytorch_models")

    # Set seeds
    seedall(seed_)

    state_dim = local_env.observation_space.shape
    action_dim = local_env.action_space.shape[0]
    max_action = float(local_env.action_space.high[0])

    # Initialize policy
    policy = DDPGAgent(state_dim, action_dim, max_action)

    replay_buffer = ReplayBuffer()

    # Evaluate untrained policy
    evaluations= [evaluate_policy(local_env, policy)]

    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    done = True
    episode_reward = None
    env_counter = 0

    while total_timesteps < max_timesteps:
        if done:
            if total_timesteps != 0:
                print(("Total T: %d Episode Num: %d Episode T: %d Reward: %f") % (
                    total_timesteps, episode_num, episode_timesteps, episode_reward))
                policy.train(replay_buffer, episode_timesteps, batch_size, discount)

            # Evaluate episode
            if timesteps_since_eval >= eval_freq:
                timesteps_since_eval %= eval_freq
                evaluations.append(evaluate_policy(local_env, policy))

                policy.save(file_name, directory="./pytorch_models")
                np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

            # Reset environment
            env_counter += 1
            obs = local_env.reset()
            done = False
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        # Select action randomly or according to policy
        if total_timesteps < start_timesteps:
            action = local_env.action_space.sample()
        else:
            action = policy.predict(np.array(obs))
            if expl_noise != 0:
                action = (action + np.random.normal(
                    0,
                    expl_noise,
                    size=local_env.action_space.shape[0])
                ).clip(-1, +1)

        # Perform action
        new_obs, reward, done, _ = local_env.step(action)

        if episode_timesteps >= env_timesteps:
            done = True
            print("DONE")

        done_bool = 0 if episode_timesteps + 1 == env_timesteps else float(done)
        episode_reward += reward

        # Store data in replay buffer
        replay_buffer.add(obs, new_obs, action, reward, done_bool)

        obs = new_obs

        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1
        print("total timesteps:", total_timesteps)

    # Final evaluation
    evaluations.append(evaluate_policy(local_env, policy))

    if save_models:
        policy.save(file_name, directory="./pytorch_models")
    np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

KeyboardInterrupt: 

# Stabilizing DDPG

As you may notice, the above model performs poorly or doesn't converge. Your job is to improve it; first in the notebook, later in the AIDO submission. This last part of the assignment consists of four sections:

**1. There are subtle, but important, bugs that have been introduced into the code above. Your job is to find them, and explain them in your `reinforcement-learning-answers.txt`. You'll want to reread the original [DQN](https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning) and [DDPG](https://arxiv.org/abs/1509.02971) papers in order to better understand the issue, but by answering the following subquestions (*please put the answers to these in the submission for full credit*), you'll be on the right track:**

   a) Read some literature on actor-critic methods, including the original [actor-critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) paper. What is an issue that you see related to *non-stationarity*? Define what _non-stationarity_ means in the context of machine learning and how it relates to actor-critic methods. In addition, give some hypotheses on why reinforcement learning is much more difficult (from an optimization perspective) than supervised learning, and how the answer to the previous question and this one are related.

   b) What role does the replay buffer play in off-policy reinforcement learning? It's most important parameter is `max_size` - how does changing this value (answer for both increasing and decreasing trends) qualitatively affect the training of the algorithm?

   c) **Challenge Question:** Briefly, explain how automatic differentiation works. In addition, expand on the difference between a single-element tensor (that `requires_grad`) and a scalar value as it relates to automatic differentiation; when do we want to backpropogate through a single-element tensor, and when do we not? Take a close look at the code and how losses are being backpropogated. On paper or your favorite drawing software, draw out the actor-critic architecture *as described in the code*, and label how the actor and critic losses are backpropogated. On your diagram, highlight the particular loss that will cause issues with the above code, and fix it.
   
For the next section, please pick **either** the theoretical or the practical pathway. If you don't have access to the necessary compute, for the exercise, please do the theoretical portion. 
   
_Theoretical Component_ 

**2. We discussed a case study of DQN in class. The original authors used quite a few tricks to get this to work. Detail some of the following, and explain what problem they solve in training the DQN:**

a) Target Networks

b) Annealed Learning Rates

c) Replay Buffer

d) Random Exploration Period

e) Preprocessing the Image


**3. Read about either [TD3](https://arxiv.org/abs/1802.09477) or [Soft Actor Critic](https://arxiv.org/abs/1801.01290); for your choice, summarize what problems they are addressing with the standard actor-critic formulation, and how they solve them**


_Practical Component_ 

**2. [Optional - if you have access to compute] Using your analysis from the reward engineering ablation, train two agents (after you've found the bugs in DDPG) - one with the standard, `gym-duckietown` reward, and another with the parameters of your choosing. Report each set of parameters, and describe qualitatively what type of behavior the agent produces.**

If you don't have the resources to actually train these agents, instead describe what types of behaviors each reward function might prioritize.

**3. [Optional - if you have access to compute] Using the instructions [here](http://docs.duckietown.org/DT19/AIDO/out/embodied_rl.html), use the saved policy files from this notebook and submit using the template submission provided through the AIDO submission. Report your best submission number (i.e the one you'd like to be graded) in `reinforcement-learning-answers.txt`**